In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='6ce270de-788d-4e07-8bae-3206860a6387'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='ajinda10_0051550143'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/keys/jiabhi_bastion'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_NAME']=os.environ['HOME']+'/work/keys/jiabhi_sliver'
os.environ['FABRIC_SLICE_PUBLIC_KEY_NAME']=os.environ['HOME']+'/work/keys/jiabhi_sliver.pub'
os.environ['FABRIC_CONFIG_LOCATION']=os.environ['HOME']+'/work/keys/fabric-ssh-config'
# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

In [2]:
FABRIC_BASTION_USERNAME = os.environ['FABRIC_BASTION_USERNAME']
FABRIC_BASTION_KEY_LOCATION = os.environ['FABRIC_BASTION_KEY_LOCATION']
FABRIC_SLICE_PRIVATE_KEY_FILE = os.environ['FABRIC_SLICE_PRIVATE_KEY_NAME']
FABRIC_BASTION_HOST = os.environ['FABRIC_BASTION_HOST']
FABRIC_CONFIG_LOCATION = os.environ['FABRIC_CONFIG_LOCATION']

In [3]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed.slice_editor import Capacities

In [4]:
manager = fablib.get_default_fablib_manager()
try:
    print(f"{manager}")
except Exception as e:
    print(f"Exception: {e}")

In [5]:
slice_name = 'jiabhi_tcp_test_slice_exp'
[site1,site2] = ['MAX', 'UTAH']
site1 = 'UTAH'
# fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'romeo'
node2_name = 'juliet'
router_name = 'router'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

image = 'default_ubuntu_20'

Sites: UTAH, UTAH


In [6]:
slice = fablib.new_slice(name=slice_name)


nodeRomeo  = slice.add_node(name=node1_name,  site=site1, cores=1, ram=4, image='default_ubuntu_20')
nodeJuliet = slice.add_node(name=node2_name, site=site1, cores=1, ram=4, image='default_ubuntu_20')
nodeRouter = slice.add_node(name=router_name, site=site1, image='default_ubuntu_20')

ifaceRomeo   = nodeRomeo.add_component(model="NIC_Basic", name="if_romeo").get_interfaces()[0]
ifaceJuliet  = nodeJuliet.add_component(model="NIC_Basic", name="if_juliet").get_interfaces()[0]
ifaceRouterR = nodeRouter.add_component(model="NIC_Basic", name="if_router_r").get_interfaces()[0]
ifaceRouterJ = nodeRouter.add_component(model="NIC_Basic", name="if_router_j").get_interfaces()[0]

netR = slice.add_l2network(name='net_r', type='L2Bridge', interfaces=[ifaceRomeo,  ifaceRouterR])
netJ = slice.add_l2network(name='net_j', type='L2Bridge', interfaces=[ifaceJuliet, ifaceRouterJ])


slice.submit()


-----------  ------------------------------------
Slice Name   jiabhi_tcp_test_slice_exp
Slice ID     57264295-a90c-4d98-a1ec-0012b3fc8f81
Slice State  StableOK
Lease End    2022-11-15 00:32:48 +0000
-----------  ------------------------------------

Retry: 15, Time: 199 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                        State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  -----------------------------------  -------  -------
794daca6-2627-41bd-aef8-d2b3c403b67c  romeo   UTAH    utah-w5.fabric-testbed.net        1      4      10  default_ubuntu_20  2001:1948:417:7:f816:3eff:feba:6a4e  Active
b7244cb8-994b-4223-bd35-c13b17b380f2  juliet  UTAH    utah-w5.fabric-testbed.net        1      4      10  default_ubuntu_20  2001:1948:417:7:f816:3eff:fea8:c5f0  Active
cc7b2376-ef70-44d7-999d-2932

'57264295-a90c-4d98-a1ec-0012b3fc8f81'

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ID                 794daca6-2627-41bd-aef8-d2b3c403b67c
Name               romeo
Cores              1
RAM                4
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               utah-w5.fabric-testbed.net
Site               UTAH
Management IP      2001:1948:417:7:f816:3eff:feba:6a4e
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:feba:6a4e
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------------------

In [8]:
# variables specific to this slice
ROMEO_IP = str(slice.get_node("romeo").get_management_ip())
ROMEO_USER =  str(slice.get_node("romeo").get_username())
ROMEO_IFACE_R = slice.get_node("romeo").get_interfaces()[0].get_os_interface()

JULIET_IP = str(slice.get_node("juliet").get_management_ip())
JULIET_USER =  str(slice.get_node("juliet").get_username())
JULIET_IFACE_J = slice.get_node("juliet").get_interfaces()[0].get_os_interface()


ROUTER_IP = str(slice.get_node("router").get_management_ip())
ROUTER_USER =  str(slice.get_node("router").get_username())
ROUTER_IFACE_J = slice.get_node("router").get_component("if_router_j").get_interfaces()[0].get_os_interface()
ROUTER_IFACE_R = slice.get_node("router").get_component("if_router_r").get_interfaces()[0].get_os_interface()

print(ROMEO_IFACE_R)
print(JULIET_IFACE_J)
print(ROUTER_IFACE_R)
print(ROUTER_IFACE_J)

ens7
ens7
ens7
ens8


In [10]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$ROMEO_USER" "$ROMEO_IP" "$ROMEO_IFACE_R"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 192.168.0.2/24 dev $5
sudo ip route add 192.168.1.0/24 via 192.168.0.1 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Nov 14 00:38:27 UTC 2022

  System load:           0.01
  Usage of /:            13.8% of 9.52GB
  Memory usage:          5%
  Swap usage:            0%
  Processes:             115
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.66
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:feba:6a4e


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 32:f3:03:c1:70:bb brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.2/

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [11]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$JULIET_USER" "$JULIET_IP" "$JULIET_IFACE_J"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 192.168.1.2/24 dev $5
sudo ip route add 192.168.0.0/24 via 192.168.1.1 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Nov 14 00:38:33 UTC 2022

  System load:           0.0
  Usage of /:            13.8% of 9.52GB
  Memory usage:          5%
  Swap usage:            0%
  Processes:             114
  Users logged in:       0
  IPv4 address for ens3: 10.20.4.211
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fea8:c5f0


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 36:9f:77:e3:8d:f9 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [12]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$ROUTER_USER" "$ROUTER_IP" "$ROUTER_IFACE_R" "$ROUTER_IFACE_J"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 192.168.0.1/24 dev $5
sudo ip addr add 192.168.1.1/24 dev $6
sudo sysctl -w net.ipv4.ip_forward=1

ip addr show dev $5
ip addr show dev $6
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Nov 14 00:38:50 UTC 2022

  System load:           0.06
  Usage of /:            13.8% of 9.52GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             140
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.125
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fef2:2934


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


net.ipv4.ip_forward = 1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 36:f1:51:27:ee:00 brd ff:ff:ff:ff:ff

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [13]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$ROUTER_USER" "$ROUTER_IP" "$ROUTER_IFACE_R" "$ROUTER_IFACE_J"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo tc qdisc del dev $5 root  
sudo tc qdisc add dev $5 root handle 1: htb default 3  
sudo tc class add dev $5 parent 1: classid 1:3 htb rate 1Mbit  
sudo tc qdisc add dev $5 parent 1:3 handle 3:  codel limit 100 target 10ms ecn

sudo tc qdisc del dev $6 root  
sudo tc qdisc add dev $6 root handle 1: htb default 3  
sudo tc class add dev $6 parent 1: classid 1:3 htb rate 1Mbit  
sudo tc qdisc add dev $6 parent 1:3 handle 3:  codel limit 100 target 10ms ecn

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Nov 14 00:39:02 UTC 2022

  System load:           0.05
  Usage of /:            13.8% of 9.52GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             138
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.125
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fef2:2934
  IPv4 address for ens7: 192.168.0.1
  IPv4 address for ens8: 192.168.1.1


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.




Pseudo-terminal will not be allocated because stdin is not a terminal.
Error: Cannot delete qdisc with handle of zero.
Error: Cannot delete qdisc with handle of zero.


In [14]:
node1 = slice.get_node(node1_name)
stdout, stderr = node1.execute("sudo apt-get update && sudo apt-get -y install iperf3")
print (stdout)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1822 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [301 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.2 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1293 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [183 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restrict

In [15]:
node2 = slice.get_node(node2_name)
stdout, stderr = node2.execute("sudo apt-get update && sudo apt-get -y install iperf3")
print (stdout)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1822 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [301 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.2 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1293 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [183 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restrict

In [16]:
stdout, stderr = node1.execute('sudo apt -y install net-tools && sudo apt-get -y install moreutils')
print(stdout)
print(stderr)

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 179 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
Fetched 196 kB in 1s (340 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63596 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20180626.aebd88e-1ubuntu1_amd64.deb ...
Unpacking net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Setting up net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Processing triggers for man-db (2.9.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libio-pty-perl libipc-run-pe

In [17]:
stdout, stderr = node2.execute('sudo apt -y install net-tools && sudo apt-get -y install moreutils')
print(stdout)
print(stderr)

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 179 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
Fetched 196 kB in 1s (341 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63596 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20180626.aebd88e-1ubuntu1_amd64.deb ...
Unpacking net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Setting up net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Processing triggers for man-db (2.9.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libio-pty-perl libipc-run-pe

In [18]:
node1.execute('sudo sysctl -w net.ipv4.tcp_ecn=1')
node2.execute('sudo sysctl -w net.ipv4.tcp_ecn=1')

('net.ipv4.tcp_ecn = 1\n', '')